### Differentiation on Linear Subspaces of G

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
alg = Algebra(1,3)
locals().update(alg.blades)

In [2]:
#2.1
F = lambda X: X**2
A = create_random_multivector(alg)
X = create_random_multivector(alg)

def differential(F, X, A, h=1e-7):
    return (F(X+h*A) - F(X))/h

max_diff(differential(F, X, A, h=1e-8), differential(F, X, A, h=1e-9))

4.440892100721072e-07

In [3]:
#2.3
A2 = create_r_blade(2, alg)
F = lambda X: P(X, A2)**2
max_diff(differential(F, X, A), differential(F, P(X,A2), A))

2.220446049250313e-09

In [4]:
#2.9
G = lambda X: (X+A2)**3
GF = lambda X: G(F(X))
max_diff(differential(GF, X, A), differential(G, F(X), differential(F, X, A)))

0.0

In [5]:
#2.12
B = create_random_multivector(alg)
FA = lambda X: differential(F, X, A)
FB = lambda X: differential(F, X, B)
FAB = differential(FA, X, B)
FBA = differential(FB, X, A)
max_diff(FAB, FBA)

0.0

In [6]:
#2.13
# Run this block several times to see dF is independent of frame
r = 4
basis = create_r_vectors(r, alg)
r_basis = reciprocal(basis)
frame = multi_frame(basis)
r_frame = multi_frame(r_basis, reverse=True)

dF = 0
for v, r in zip(frame, r_frame):
    dF += r * differential(F, X, v)
dF

0.0808 + 3.97 𝐞₁ + 0.604 𝐞₂ + 2.02 𝐞₃ + 2.35 𝐞₄ + 0.962 𝐞₁₂ + 0.768 𝐞₁₃ + 0.543 𝐞₁₄ + -0.374 𝐞₂₃ + -0.486 𝐞₂₄ + -0.178 𝐞₃₄ + 9.08e-10 𝐞₁₂₃ + -6.69e-08 𝐞₁₂₄ + -5.72e-08 𝐞₁₃₄ + -9.73e-09 𝐞₂₃₄ + 4.65e-09 𝐞₁₂₃₄

In [7]:
# For F defined on A2, we can restrict our bases
# choose a nonorthonormal-to-A2-vector e.g. e1
v1 = P(e1, A2)
# contract the parallel part
v2 = v1 | A2
frame = multi_frame([v1, v2])
r_frame = multi_frame(reciprocal([v1, v2]), reverse=True)
dF = 0
for v, r in zip(frame, r_frame):
    dF += r * (differential(F, X, v))
dF

0.0808 + 3.97 𝐞₁ + 0.604 𝐞₂ + 2.02 𝐞₃ + 2.35 𝐞₄ + 0.962 𝐞₁₂ + 0.768 𝐞₁₃ + 0.543 𝐞₁₄ + -0.374 𝐞₂₃ + -0.486 𝐞₂₄ + -0.178 𝐞₃₄ + -1.28e-10 𝐞₁₂₃ + -2.25e-10 𝐞₁₂₄ + -4.05e-10 𝐞₁₃₄ + 5.88e-11 𝐞₂₃₄ + 8.24e-11 𝐞₁₂₃₄

In [8]:
#2.18 Maybe a better difference scheme, or a better frame leads to better result
#  It suggests always replace d_F_ with dF in computation for better accuracy/stability
d_F_ = 0
frame = multi_frame([v1, v2])
r_frame = multi_frame(reciprocal([v1, v2]), reverse=True)
F_ = lambda A: differential(F, X, A)
for v, r in zip(frame, r_frame):
    d_F_ += r * differential(F_, A, v)

max_diff(d_F_, dF), d_F_, dF

(0.05677544513729993,
 0.0804 + 3.91 𝐞₁ + 0.613 𝐞₂ + 2.01 𝐞₃ + 2.32 𝐞₄ + 0.969 𝐞₁₂ + 0.768 𝐞₁₃ + 0.544 𝐞₁₄ + -0.374 𝐞₂₃ + -0.482 𝐞₂₄ + -0.172 𝐞₃₄ + 0.000813 𝐞₁₂₃ + 0.000536 𝐞₁₂₄ + 0.000721 𝐞₁₃₄ + 0.00047 𝐞₂₃₄ + 0.00602 𝐞₁₂₃₄,
 0.0808 + 3.97 𝐞₁ + 0.604 𝐞₂ + 2.02 𝐞₃ + 2.35 𝐞₄ + 0.962 𝐞₁₂ + 0.768 𝐞₁₃ + 0.543 𝐞₁₄ + -0.374 𝐞₂₃ + -0.486 𝐞₂₄ + -0.178 𝐞₃₄ + -1.28e-10 𝐞₁₂₃ + -2.25e-10 𝐞₁₂₄ + -4.05e-10 𝐞₁₃₄ + 5.88e-11 𝐞₂₃₄ + 8.24e-11 𝐞₁₂₃₄)

In [9]:
# 2.19
def adjoint(F, X, A, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = multi_frame(alg.frame)
        r_frame = reci_frame(alg.frame)
    _F = 0    
    for v, r in zip(frame, r_frame):
        _F += r * alg.sp(differential(F, X, v), A)
    return _F

_F = adjoint(F, X, A, alg)
max_diff(B.sp(_F), differential(F, X, B).sp(A))


1.2466149779122304e-09

In [10]:
#2.20 F defined on A2
max_diff(P(_F, A2), _F)

5.591634710810922e-08

In [11]:
#2.21
def derivative(F, X, alg, grade=None, frame=None, r_frame=None):
    if not frame:
        if grade or (grade == 0):
            frame = r_vector_frame(alg.frame, grade)
            r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
        else:
            frame = multi_frame(alg.frame)
            r_frame = reci_frame(alg.frame)
    dF = 0
    for v, r in zip(frame, r_frame):
        dF += r * (differential(F, X, v))
    return dF

[max_diff(_F.grade(r), derivative(lambda X: F(X).sp(A), X, alg, r)) for r in range(0, 5)]

[3.3918368114171926e-10,
 8.538025841886565e-10,
 8.274746721159687e-10,
 9.02070417709378e-10,
 0.0]

In [12]:
#2.24
r = 2
_sum = derivative(lambda X: F(X), X, alg) + derivative(lambda X: G(X), X, alg)
max_diff(derivative(lambda X: F(X) + G(X), X, alg), _sum)

3.306816154236003e-09

In [13]:
#2.25a
_prod = derivative(lambda X: F(X), X, alg)*G(X) + derivative(lambda Y:F(X)*G(Y), X, alg)
max_diff(derivative(lambda X: F(X)*G(X), X, alg), _prod)

1.8898704645664566e-06

In [14]:
#2.25b
grade = 2
frame = r_vector_frame(alg.frame, grade)
r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

5.633347193167992e-07

In [15]:
#2.25b
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

1.8887221617802652e-06

In [16]:
#2.25b
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

1.8887221617802652e-06

In [17]:
#2.26
r = 2
(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

(-1.51 + -0.0226 𝐞₁ + -0.0127 𝐞₂ + -0.019 𝐞₃ + -0.0186 𝐞₄ + 1.67 𝐞₁₂ + 1.33 𝐞₁₃ + 0.941 𝐞₁₄ + -0.648 𝐞₂₃ + -0.843 𝐞₂₄ + -0.308 𝐞₃₄ + -4.13e-10 𝐞₁₂₃ + -9.92e-10 𝐞₁₂₄ + 1.08e-08 𝐞₁₃₄ + -8.11e-09 𝐞₂₃₄ + -2.86e-08 𝐞₁₂₃₄,
 -1.5 + -0.0251 𝐞₁ + -0.0119 𝐞₂ + -0.0186 𝐞₃ + -0.0202 𝐞₄ + 1.76 𝐞₁₂ + 1.38 𝐞₁₃ + 1.02 𝐞₁₄ + -0.645 𝐞₂₃ + -0.944 𝐞₂₄ + -0.424 𝐞₃₄ + 9.26e-05 𝐞₁₂₃ + -0.000328 𝐞₁₂₄ + -0.000192 𝐞₁₃₄ + -0.000609 𝐞₂₃₄ + 0.0305 𝐞₁₂₃₄)

In [18]:
#2.28a
(
    max_diff(P(A, A2), differential(lambda X: P(X, A2), X, A)),
    max_diff(P(A, A2), derivative(lambda X: P(X, A2).sp(A), X, alg))
)

(3.1353304397185866e-09, 1.4699959027808518e-09)

In [19]:
#2.28b
max_diff(P(A, A2).reverse(), differential(lambda X: P(X.reverse(), A2), X, A))

1.1788404141732656e-09

In [20]:
#2.29
derivative(lambda X: X, X, alg)

16.0

In [35]:
#2.30
alg = Algebra(1)
X = create_random_multivector(alg)
derivative(lambda X: X * X.reverse(), X, alg), 4*X.reverse()

(2.23 + 2.24 𝐞₁, 2.23 + 2.24 𝐞₁)

In [21]:
#2.30
alg = Algebra(2)
X = create_random_multivector(alg)
derivative(lambda X: X * X.reverse(), X, alg), 6*X.reverse()

(2.1 + 6.0 𝐞₁ + 5.48 𝐞₂ + -1.06 𝐞₁₂, 2.1 + 6.0 𝐞₁ + 5.48 𝐞₂ + -1.06 𝐞₁₂)

In [22]:
#2.30
alg = Algebra(3)
X = create_random_multivector(alg)
derivative(lambda X: X * X.reverse(), X, alg), 8*X.reverse()

(2.76 + 0.838 𝐞₁ + 4.53 𝐞₂ + 4.07 𝐞₃ + -4.33 𝐞₁₂ + -2.63 𝐞₁₃ + -1.65 𝐞₂₃ + -1.5 𝐞₁₂₃,
 2.76 + 0.838 𝐞₁ + 4.53 𝐞₂ + 4.07 𝐞₃ + -4.33 𝐞₁₂ + -2.63 𝐞₁₃ + -1.65 𝐞₂₃ + -1.5 𝐞₁₂₃)

In [23]:
#2.30
alg = Algebra(4)
X = create_random_multivector(alg)
derivative(lambda X: X * X.reverse(), X, alg), 12*X.reverse()

(0.409 + 6.02 𝐞₁ + 9.65 𝐞₂ + 6.14 𝐞₃ + 11.2 𝐞₄ + -7.77 𝐞₁₂ + -6.0 𝐞₁₃ + -3.6 𝐞₁₄ + -7.81 𝐞₂₃ + -1.02 𝐞₂₄ + -1.3 𝐞₃₄ + -4.92 𝐞₁₂₃ + -6.7 𝐞₁₂₄ + -7.67 𝐞₁₃₄ + -11.2 𝐞₂₃₄ + 11.5 𝐞₁₂₃₄,
 0.409 + 6.02 𝐞₁ + 9.65 𝐞₂ + 6.14 𝐞₃ + 11.2 𝐞₄ + -7.77 𝐞₁₂ + -6.0 𝐞₁₃ + -3.6 𝐞₁₄ + -7.81 𝐞₂₃ + -1.02 𝐞₂₄ + -1.3 𝐞₃₄ + -4.92 𝐞₁₂₃ + -6.7 𝐞₁₂₄ + -7.67 𝐞₁₃₄ + -11.2 𝐞₂₃₄ + 11.5 𝐞₁₂₃₄)

In [24]:
#2.30
alg = Algebra(5)
X = create_random_multivector(alg)
derivative(lambda X: X * X.reverse(), X, alg), 24*X.reverse()

(8.72 + 2.81 𝐞₁ + 11.9 𝐞₂ + 14.1 𝐞₃ + 0.354 𝐞₄ + 8.97 𝐞₅ + -12.8 𝐞₁₂ + -8.18 𝐞₁₃ + -9.32 𝐞₁₄ + -8.84 𝐞₁₅ + -4.71 𝐞₂₃ + -20.1 𝐞₂₄ + -23.0 𝐞₂₅ + -2.81 𝐞₃₄ + -7.67 𝐞₃₅ + -16.3 𝐞₄₅ + -20.2 𝐞₁₂₃ + -0.0787 𝐞₁₂₄ + -1.01 𝐞₁₂₅ + -15.3 𝐞₁₃₄ + -9.71 𝐞₁₃₅ + -19.4 𝐞₁₄₅ + -21.1 𝐞₂₃₄ + -15.0 𝐞₂₃₅ + -23.5 𝐞₂₄₅ + -5.09 𝐞₃₄₅ + 10.4 𝐞₁₂₃₄ + 11.7 𝐞₁₂₃₅ + 17.6 𝐞₁₂₄₅ + 22.3 𝐞₁₃₄₅ + 23.0 𝐞₂₃₄₅ + 21.9 𝐞₁₂₃₄₅,
 8.72 + 2.81 𝐞₁ + 11.9 𝐞₂ + 14.1 𝐞₃ + 0.354 𝐞₄ + 8.97 𝐞₅ + -12.8 𝐞₁₂ + -8.18 𝐞₁₃ + -9.32 𝐞₁₄ + -8.84 𝐞₁₅ + -4.71 𝐞₂₃ + -20.1 𝐞₂₄ + -23.0 𝐞₂₅ + -2.81 𝐞₃₄ + -7.67 𝐞₃₅ + -16.3 𝐞₄₅ + -20.2 𝐞₁₂₃ + -0.0787 𝐞₁₂₄ + -1.01 𝐞₁₂₅ + -15.3 𝐞₁₃₄ + -9.71 𝐞₁₃₅ + -19.4 𝐞₁₄₅ + -21.1 𝐞₂₃₄ + -15.0 𝐞₂₃₅ + -23.5 𝐞₂₄₅ + -5.09 𝐞₃₄₅ + 10.4 𝐞₁₂₃₄ + 11.7 𝐞₁₂₃₅ + 17.6 𝐞₁₂₄₅ + 22.3 𝐞₁₃₄₅ + 23.0 𝐞₂₃₄₅ + 21.9 𝐞₁₂₃₄₅)

In [25]:
#2.30
alg = Algebra(6)
X = create_random_multivector(alg)
derivative(lambda X: X * X.reverse(), X, alg), 56*X.reverse()

(16.9 + 26.6 𝐞₁ + 9.24 𝐞₂ + 54.9 𝐞₃ + 54.6 𝐞₄ + 34.7 𝐞₅ + 53.8 𝐞₆ + -19.0 𝐞₁₂ + -35.3 𝐞₁₃ + -54.2 𝐞₁₄ + -41.0 𝐞₁₅ + -52.2 𝐞₁₆ + -53.2 𝐞₂₃ + -17.3 𝐞₂₄ + -11.5 𝐞₂₅ + -33.5 𝐞₂₆ + -19.9 𝐞₃₄ + -36.1 𝐞₃₅ + -47.8 𝐞₃₆ + -38.5 𝐞₄₅ + -20.6 𝐞₄₆ + -49.7 𝐞₅₆ + -4.65 𝐞₁₂₃ + -17.6 𝐞₁₂₄ + -14.3 𝐞₁₂₅ + -19.2 𝐞₁₂₆ + -12.9 𝐞₁₃₄ + -19.1 𝐞₁₃₅ + -30.8 𝐞₁₃₆ + -3.15 𝐞₁₄₅ + -10.5 𝐞₁₄₆ + -47.0 𝐞₁₅₆ + -37.8 𝐞₂₃₄ + -14.1 𝐞₂₃₅ + -34.3 𝐞₂₃₆ + -29.2 𝐞₂₄₅ + -13.1 𝐞₂₄₆ + -24.5 𝐞₂₅₆ + -27.8 𝐞₃₄₅ + -25.4 𝐞₃₄₆ + -46.2 𝐞₃₅₆ + -47.6 𝐞₄₅₆ + 41.5 𝐞₁₂₃₄ + 38.3 𝐞₁₂₃₅ + 41.7 𝐞₁₂₃₆ + 50.2 𝐞₁₂₄₅ + 24.6 𝐞₁₂₄₆ + 46.9 𝐞₁₂₅₆ + 30.0 𝐞₁₃₄₅ + 45.2 𝐞₁₃₄₆ + 4.84 𝐞₁₃₅₆ + 21.3 𝐞₁₄₅₆ + 49.0 𝐞₂₃₄₅ + 4.83 𝐞₂₃₄₆ + 19.5 𝐞₂₃₅₆ + 14.9 𝐞₂₄₅₆ + 55.8 𝐞₃₄₅₆ + 18.6 𝐞₁₂₃₄₅ + 12.0 𝐞₁₂₃₄₆ + 35.6 𝐞₁₂₃₅₆ + 17.3 𝐞₁₂₄₅₆ + 28.4 𝐞₁₃₄₅₆ + 53.0 𝐞₂₃₄₅₆ + -3.87 𝐞₁₂₃₄₅₆,
 16.9 + 26.6 𝐞₁ + 9.24 𝐞₂ + 54.9 𝐞₃ + 54.6 𝐞₄ + 34.7 𝐞₅ + 53.8 𝐞₆ + -19.0 𝐞₁₂ + -35.3 𝐞₁₃ + -54.2 𝐞₁₄ + -41.0 𝐞₁₅ + -52.2 𝐞₁₆ + -53.2 𝐞₂₃ + -17.3 𝐞₂₄ + -11.5 𝐞₂₅ + -33.5 𝐞₂₆ + -19.9 𝐞₃₄ + -36.1 𝐞₃₅ 

The formula 2.2.30 seems to be wrong, instead the formula should be given by:
$$ \partial  X\tilde{X} = \.{\partial} \.{X} \tilde{X} + \.{\partial} X \.{\tilde{X}} = 2^n \tilde{X} + {\Gamma}_n \tilde{X}$$
where ${\Gamma}_n$ with $n \leq 6$ are suggests to be [2,2,0,-4,-8,-8] by the above and below numerical tests:

In [26]:
for n in range(1,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda Y: X*Y.reverse(), X, alg)
    print(n, dXXr[0]/X[0])

1 2.0000000006869167
2 2.000000001599846
3 -7.485996790207376e-10
4 -3.9999948785211963
5 -8.000000012388812
6 -7.999999868486835


In [37]:
#2.37a
from scipy.special import comb
for n, r in [(5,2), (6,2), (1,1), (1,0), (2,0)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    print(derivative(lambda X: X, X, alg, r), comb(n, r, exact=True))

10.0 10
15.0 15
1.0 1
1.0 1
1.0 1


In [39]:
#2.38a
def gamma(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    print(derivative(lambda X: Ar*X, X, alg, s)) 
    print(gamma(n, r, s)*Ar)

0.572 𝐞₁₂ + 0.0994 𝐞₁₃ + 0.448 𝐞₁₄ + 0.179 𝐞₁₅ + 0.569 𝐞₂₃ + 0.552 𝐞₂₄ + 0.677 𝐞₂₅ + 0.289 𝐞₃₄ + 0.623 𝐞₃₅ + 0.474 𝐞₄₅
0.572 𝐞₁₂ + 0.0994 𝐞₁₃ + 0.448 𝐞₁₄ + 0.179 𝐞₁₅ + 0.569 𝐞₂₃ + 0.552 𝐞₂₄ + 0.677 𝐞₂₅ + 0.289 𝐞₃₄ + 0.623 𝐞₃₅ + 0.474 𝐞₄₅
-1.94 𝐞₁₂ + -2.72 𝐞₁₃ + -0.153 𝐞₁₄ + -3.98 𝐞₁₅ + -2.23 𝐞₁₆ + -2.25 𝐞₂₃ + -2.09 𝐞₂₄ + -2.6 𝐞₂₅ + -3.92 𝐞₂₆ + -1.3 𝐞₃₄ + -3.95 𝐞₃₅ + -3.15 𝐞₃₆ + -3.78 𝐞₄₅ + -1.24 𝐞₄₆ + -0.519 𝐞₅₆
-1.94 𝐞₁₂ + -2.72 𝐞₁₃ + -0.153 𝐞₁₄ + -3.98 𝐞₁₅ + -2.23 𝐞₁₆ + -2.25 𝐞₂₃ + -2.09 𝐞₂₄ + -2.6 𝐞₂₅ + -3.92 𝐞₂₆ + -1.3 𝐞₃₄ + -3.95 𝐞₃₅ + -3.15 𝐞₃₆ + -3.78 𝐞₄₅ + -1.24 𝐞₄₆ + -0.519 𝐞₅₆
0.231 𝐞₁
0.231 𝐞₁
0.366
0.366
1.73
1.73


In [44]:
#2.38b
n = 5
s = 2
alg = Algebra(n)
X = create_random_multivector(alg)
A = create_random_multivector(alg)
derivative(lambda X: A*X, X, alg, s), sum(gamma(n, r, s)*A.grade(r) for r in range(0, n+1))


(9.38 + 1.26 𝐞₁ + 1.23 𝐞₂ + 1.71 𝐞₃ + 0.488 𝐞₄ + 0.657 𝐞₅ + -1.93 𝐞₁₂ + -0.637 𝐞₁₃ + -1.03 𝐞₁₄ + -0.884 𝐞₁₅ + -1.81 𝐞₂₃ + -1.45 𝐞₂₄ + -0.978 𝐞₂₅ + -1.01 𝐞₃₄ + -0.802 𝐞₃₅ + -0.678 𝐞₄₅ + -0.897 𝐞₁₂₃ + -0.556 𝐞₁₂₄ + -0.655 𝐞₁₂₅ + -1.64 𝐞₁₃₄ + -1.77 𝐞₁₃₅ + -1.24 𝐞₁₄₅ + -0.551 𝐞₂₃₄ + -1.06 𝐞₂₃₅ + -0.84 𝐞₂₄₅ + -1.53 𝐞₃₄₅ + 0.569 𝐞₁₂₃₄ + 0.947 𝐞₁₂₃₅ + 1.65 𝐞₁₂₄₅ + 1.23 𝐞₁₃₄₅ + 0.958 𝐞₂₃₄₅ + 7.87 𝐞₁₂₃₄₅,
 9.38 + 1.26 𝐞₁ + 1.23 𝐞₂ + 1.71 𝐞₃ + 0.488 𝐞₄ + 0.657 𝐞₅ + -1.93 𝐞₁₂ + -0.637 𝐞₁₃ + -1.03 𝐞₁₄ + -0.884 𝐞₁₅ + -1.81 𝐞₂₃ + -1.45 𝐞₂₄ + -0.978 𝐞₂₅ + -1.01 𝐞₃₄ + -0.802 𝐞₃₅ + -0.678 𝐞₄₅ + -0.897 𝐞₁₂₃ + -0.556 𝐞₁₂₄ + -0.655 𝐞₁₂₅ + -1.64 𝐞₁₃₄ + -1.77 𝐞₁₃₅ + -1.24 𝐞₁₄₅ + -0.551 𝐞₂₃₄ + -1.06 𝐞₂₃₅ + -0.84 𝐞₂₄₅ + -1.53 𝐞₃₄₅ + 0.569 𝐞₁₂₃₄ + 0.947 𝐞₁₂₃₅ + 1.65 𝐞₁₂₄₅ + 1.23 𝐞₁₃₄₅ + 0.958 𝐞₂₃₄₅ + 7.87 𝐞₁₂₃₄₅)

Let's try construct $\Gamma_n$ from $\Gamma_s^r$

In [40]:
#2.38a reverse version
def gamma_rev(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k+s*(s-1)/2)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    print(derivative(lambda X: Ar*X.reverse(), X, alg, s)) 
    print(gamma_rev(n, r, s)*Ar)

0.0452 𝐞₁₂ + 0.468 𝐞₁₃ + 0.428 𝐞₁₄ + 0.358 𝐞₁₅ + 0.405 𝐞₂₃ + 0.356 𝐞₂₄ + 0.272 𝐞₂₅ + 0.412 𝐞₃₄ + 0.64 𝐞₃₅ + 0.367 𝐞₄₅
0.0452 𝐞₁₂ + 0.468 𝐞₁₃ + 0.428 𝐞₁₄ + 0.358 𝐞₁₅ + 0.405 𝐞₂₃ + 0.356 𝐞₂₄ + 0.272 𝐞₂₅ + 0.412 𝐞₃₄ + 0.64 𝐞₃₅ + 0.367 𝐞₄₅
2.88 𝐞₁₂ + 2.03 𝐞₁₃ + 3.57 𝐞₁₄ + 2.61 𝐞₁₅ + 0.732 𝐞₁₆ + 3.91 𝐞₂₃ + 0.951 𝐞₂₄ + 0.242 𝐞₂₅ + 3.34 𝐞₂₆ + 2.6 𝐞₃₄ + 0.657 𝐞₃₅ + 3.79 𝐞₃₆ + 3.65 𝐞₄₅ + 3.13 𝐞₄₆ + 0.928 𝐞₅₆
2.88 𝐞₁₂ + 2.03 𝐞₁₃ + 3.57 𝐞₁₄ + 2.61 𝐞₁₅ + 0.732 𝐞₁₆ + 3.91 𝐞₂₃ + 0.951 𝐞₂₄ + 0.242 𝐞₂₅ + 3.34 𝐞₂₆ + 2.6 𝐞₃₄ + 0.657 𝐞₃₅ + 3.79 𝐞₃₆ + 3.65 𝐞₄₅ + 3.13 𝐞₄₆ + 0.928 𝐞₅₆
0.643 𝐞₁
0.643 𝐞₁
0.35
0.35
0.167
0.167


In [54]:
for n in range(7):
    r_parts = np.zeros(n+1)
    for s in range(0, n+1):
        for r in range(0, n+1):
            r_parts[r] += (-1)**(r*(r-1)/2)*gamma_rev(n, r, s) # one more reversion
    print(r_parts)

[1.]
[2. 2.]
[2. 2. 2.]
[0. 0. 0. 0.]
[-4. -4. -4. -4. -4.]
[-8. -8. -8. -8. -8. -8.]
[-8. -8. -8. -8. -8. -8. -8.]


The coefficients of each grade match due to the reversion-symmetry of gamma_rev.
Thus we get $\Gamma_n$:

In [56]:
def gamma_n(n):
    return sum(gamma_rev(n, 0, s) for s in range(0, n+1))
[gamma_n(n) for n in range(7)]

[1.0, 2.0, 2.0, 0.0, -4.0, -8.0, -8.0]